In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

In [ ]:
kommentare = []

class Kommentar:
    def __init__(self, url, datum, text):
        self.url = url
        self.datum = datum
        self.text = text

In [ ]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [ ]:
start = "https://www.tagesanzeiger.ch/"
driver.get(start)

cookies = pickle.load(open("tagi_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [ ]:
with open("tagesanzeiger_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)

In [ ]:
driver.get("https://www.tagesanzeiger.ch/das-rosige-lockdown-szenario-der-corona-taskforce-385281546789")

In [ ]:
def inhalter():
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        block = soup.find_all(class_="comment-item")

        for i in block:
            sa = i.find(class_="RelativeDateTime_root__29Kdb")
            datum = sa.get_text()

            h = i.find(class_="text")
            h = h.get_text()
            text = h.replace("\n", "")


            crawled_kom = Kommentar(page, datum, text)
            kommentare.append(crawled_kom)
    except TimeoutError:
        print("Time Out")

In [ ]:
for page in ueb_pages:
    time.sleep(1)
    driver.get(page)
    cookies = pickle.load(open("tagi_cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    try:
        x = driver.find_element_by_xpath("//button[contains(.,'Alle Kommentare anzeigen')]")
        time.sleep(1)
        x.click()
        inhalter()
    except NoSuchElementException:
        print("No such element - we go on")

In [ ]:
with open('tagi_kom.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter_kom = csv.writer(csvfile, delimiter='|', quotechar='"')
                                
    for element in kommentare:
        blogwriter_kom.writerow( [element.url, element.datum, element.text] )

    